In [1]:
# %load LOOCV.py
%matplotlib
from matplotlib import pyplot as plt
import lasio
from sklearn import preprocessing
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import openpyxl
from sklearn.svm import SVR
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import tree
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
import random
from matplotlib.font_manager import *
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [2]:
from model import one_cnn_model, three_cnn_model, bp_model, LinearRegression_model1, LinearRegression_model2
import model
import imp
imp.reload(model)
from model import one_cnn_model, three_cnn_model, bp_model, LinearRegression_model1, LinearRegression_model2

In [ ]:
from DataSet import dataset_lab
import DataSet
import imp
imp.reload(DataSet)
from DataSet import dataset_lab

In [5]:
from Normalization import Normalization, Anti_Normalization, Normalization_lab, Anti_Normalization_lab
import Normalization
import imp
imp.reload(Normalization)
from Normalization import Normalization, Anti_Normalization, Normalization_lab, Anti_Normalization_lab

In [6]:
# 读取初始化相关数据
b = 6

In [ ]:
data, log_GAS = dataset_lab(b)
print(data.shape)

In [239]:
shuffle_data = np.hstack((data, log_GAS))

In [8]:
np.random.shuffle(shuffle_data)
print(shuffle_data.shape)

(201, 5)


In [9]:
model = LinearRegression()
shuffle_RHOB = shuffle_data[:, 2].reshape(-1, 1)
shuffle_GAS = shuffle_data[:, 3].reshape(-1, 1)
model.fit(shuffle_RHOB, shuffle_GAS)
score = model.score(shuffle_RHOB, shuffle_GAS)
ev = model.coef_  # 回归系数
inter = model.intercept_
shuffle_LR_GAS = model.predict(shuffle_RHOB)
shuffle_LR_GAS = shuffle_LR_GAS.reshape(-1, 1)
#     print("yhat:", yhat)
#     print("yhat_array[test_index][0]:", yhat_array[test_index][0])
print("ev:", ev)
print("inter:", inter)

ev: [[-10.56499268]]
inter: [29.40776038]


In [10]:
Onefil_Aim = []
Onefil_Para = []
Onefil_log_GAS = []
shuffle_Aim = shuffle_data[:, 3].reshape(-1, 1)
shuffle_Para_VVR = shuffle_data[:, 0: 3].reshape(-1, 3)
shuffle_log_GAS = shuffle_data[:, 4].reshape(-1, 1)
b = 1.6
for i in range(shuffle_Aim.shape[0]):
    if(-b< (shuffle_Aim[i] - shuffle_LR_GAS[i]) < b):
        Onefil_Aim.append(shuffle_Aim[i])
        Onefil_Para.append(shuffle_Para_VVR[i])
        Onefil_log_GAS.append(shuffle_log_GAS[i])
Onefil_Aim = np.array(Onefil_Aim).reshape(-1, 1)
Onefil_Para = np.array(Onefil_Para)
Onefil_data = np.hstack((Onefil_Para, Onefil_Aim))
print(Onefil_data.shape)

(179, 4)


In [11]:
Onefil_NL_data = Normalization_lab(Onefil_data, 11)

(179, 4)


cnn

In [228]:
loo = LeaveOneOut()
Onefil_cnn =[]
Onefil_NL_input_data = Onefil_NL_data[:, 0: 3].reshape(-1, 1, 3)
Onefil_NL_output_data = Onefil_NL_data[:, 3].reshape(-1, 1, 1)
loo.get_n_splits(Onefil_NL_input_data)
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]
    
    Onefil_yhat, model, history = three_cnn_model(NL_train_X_data, NL_train_Y_data, NL_test_X_data, NL_test_Y_data, 128)
    
    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_cnn.append(Onefil_yhat)
Onefil_cnn = np.array(Onefil_cnn)

rf

In [250]:
Onefil_NL_input_data = np.hstack((Onefil_NL_data[:, 0:3])).reshape(-1, 3)
Onefil_NL_output_data = Onefil_NL_data[0:,3].reshape(-1, 1)

In [194]:
loo = LeaveOneOut()
loo.get_n_splits(Onefil_NL_input_data)
Onefil_rf =[]
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]

    NL_train_X_data, NL_train_Y_data, NL_test_X_data = NL_train_X_data.reshape(-1, data.shape[1] - 1), NL_train_Y_data.reshape(-1,1), NL_test_X_data.reshape(-1, data.shape[1] - 1)
    rfr = RandomForestRegressor(n_estimators = 32, max_depth = 10, min_samples_leaf = 2, random_state=20)
    Onefil_yhat = rfr.fit(NL_train_X_data,NL_train_Y_data).predict(NL_test_X_data).reshape(-1, 1)

    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_rf.append(Onefil_yhat)
Onefil_rf= np.array(Onefil_rf).reshape(-1, 1)
print(Onefil_rf.shape)

(179, 1)


DT

In [327]:
loo = LeaveOneOut()
loo.get_n_splits(Onefil_NL_input_data)
Onefil_dt =[]
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]

    NL_train_X_data, NL_train_Y_data, NL_test_X_data = NL_train_X_data.reshape(-1, data.shape[1] - 1), NL_train_Y_data.reshape(-1,1), NL_test_X_data.reshape(-1, data.shape[1] - 1)
    dtr = tree.DecisionTreeRegressor(max_depth = 10, min_samples_leaf = 7, random_state=13)
    Onefil_yhat = dtr.fit(NL_train_X_data, NL_train_Y_data).predict(NL_test_X_data).reshape(-1, 1)

    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_dt.append(Onefil_yhat)
Onefil_dt= np.array(Onefil_dt).reshape(-1, 1)

SVR

In [251]:
loo = LeaveOneOut()
loo.get_n_splits(Onefil_NL_input_data)
Onefil_svr =[]
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]

    NL_train_X_data, NL_train_Y_data, NL_test_X_data = NL_train_X_data.reshape(-1, data.shape[1] - 1), NL_train_Y_data.reshape(-1,1), NL_test_X_data.reshape(-1, data.shape[1] - 1)
    svr_rbf = SVR(kernel='rbf', C=1e2, gamma=0.4)
    Onefil_yhat = svr_rbf.fit(NL_train_X_data, NL_train_Y_data).predict(NL_test_X_data).reshape(-1, 1)

    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_svr.append(Onefil_yhat)
#     print("yhat:", yhat)
#     print("yhat_array[test_index][0]:", yhat_array[test_index][0])
Onefil_svr= np.array(Onefil_svr).reshape(-1, 1)
print(Onefil_svr.shape)

(179, 1)


LR

In [258]:
loo = LeaveOneOut()
loo.get_n_splits(Onefil_NL_input_data)
Onefil_lr =[]
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]

    NL_train_X_data, NL_train_Y_data, NL_test_X_data = NL_train_X_data.reshape(-1, data.shape[1] - 1), NL_train_Y_data.reshape(-1,1), NL_test_X_data.reshape(-1, data.shape[1] - 1)
    Onefil_yhat, model, score, ev  = LinearRegression_model1(NL_train_X_data, NL_train_Y_data, NL_test_X_data, NL_test_Y_data)


    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_lr.append(Onefil_yhat)
#     print("yhat:", yhat)
#     print("yhat_array[test_index][0]:", yhat_array[test_index][0])
Onefil_lr= np.array(Onefil_lr).reshape(-1, 1)

bp

In [196]:
loo = LeaveOneOut()
loo.get_n_splits(Onefil_NL_input_data)
Onefil_bp =[]
for train_index, test_index in loo.split(Onefil_NL_input_data):
    NL_train_X_data, NL_test_X_data = Onefil_NL_input_data[train_index], Onefil_NL_input_data[test_index]
    NL_train_Y_data, NL_test_Y_data = Onefil_NL_output_data[train_index], Onefil_NL_output_data[test_index]

    NL_train_X_data, NL_train_Y_data, NL_test_X_data = NL_train_X_data.reshape(-1, data.shape[1] - 1), NL_train_Y_data.reshape(-1,1), NL_test_X_data.reshape(-1, data.shape[1] - 1)
    Onefil_yhat, model, history = bp_model(NL_train_X_data, NL_train_Y_data, NL_test_X_data, NL_test_Y_data)

    Onefil_yhat = Anti_Normalization_lab(Onefil_data, NL_test_X_data, Onefil_yhat)
    Onefil_bp.append(Onefil_yhat)
#     print("yhat:", yhat)
#     print("yhat_array[test_index][0]:", yhat_array[test_index][0])
Onefil_bp= np.array(Onefil_bp).reshape(-1, 1)

In [1]:
a = Onefil_svr / 2 +  Onefil_lr.reshape(-1, 1) /2
Onefil_Aim = Onefil_data[:,2].reshape(-1, )
print(a.shape)
print(Onefil_Aim.shape)
# print("a_Aim:", np.hstack((Onefil_Aim, Onefil_svr.reshape(-1, 1))))
# int('mse: ', mean_squared_error())
print('R2: ', np.corrcoef(Onefil_lr.reshape(-1, ), Onefil_Aim))
print('R2: ', r2_score(Onefil_Aim, a))
print('mse: ', mean_squared_error(Onefil_Aim, a))

NameError: name 'Onefil_svr' is not defined

测井曲线

In [318]:
plt.figure(figsize=(20, 10))
plt.plot(Onefil_Aim, '-', linewidth=2, label = 'Core_GAS')
plt.plot(Onefil_log_GAS, '-', linewidth=2, label = 'Log_GAS')
plt.xticks(fontsize=20);
plt.yticks(fontsize=20);
plt.legend(loc = 'upper right', fontsize = 20)
plt.show

<function matplotlib.pyplot.show>

预测曲线

In [300]:
fig_1 = plt.figure(figsize=(20, 10))  # 作图画布大小(每循环一次，就会建立新的画布）
# for f in range(3):
#     ax = fig_2.add_subplot(1, 4, f + 1)  # 图布一行len(a)列，画在第f+1块
#     ax.plot(Para_VVR[train_data_length:, f], depth)
#     ax.set_xlabel(df_Para.keys()[f])
#     ax.xaxis.tick_top()
#     ax.invert_yaxis()
#     plt.tick_params(labelsize=6)
# ax = fig_2.add_subplot(1, 1, 1)
# ax.plot(Para_VVR[:, 2], a)
ax = fig_1.add_subplot(9, 1,1)
ax.plot(data[:,0], '-', linewidth=2, label='DTS')
ax.set_ylabel('Para_DTS', fontsize=13)
ax = fig_1.add_subplot(9, 1, 2)
ax.plot(data[:,1], '-', linewidth=2, label='DT')
ax.set_ylabel('Para_DT', fontsize=13)
ax = fig_1.add_subplot(9, 1, 3)
ax.plot(data[:,2], '-', linewidth=2, label='RHOB')
ax.set_ylabel('Para_RHOB', fontsize=13)
ax = fig_1.add_subplot( 9, 1,4)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_svr, '-', linewidth=2, label='SVR_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('SVR', fontsize=13)
ax = fig_1.add_subplot(9, 1, 5)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_dt, '-', linewidth=2, label='DT_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('DT',fontsize=13)
ax = fig_1.add_subplot(9, 1, 6)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_rf, '-', linewidth=2, label='RF_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('RF', fontsize=13)
ax = fig_1.add_subplot(9, 1, 7)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_bp, '-', linewidth=2, label='BP_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('BP', fontsize=13)
ax = fig_1.add_subplot(9, 1, 8)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_cnn.reshape(-1, 1), '-', linewidth=2, label='CNN_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('CNN', fontsize=13)
ax = fig_1.add_subplot(9, 1, 9)
ax.plot(Onefil_Aim, '-', linewidth=2, label='Core_value')
ax.plot(Onefil_lr.reshape(-1, 1), '-', linewidth=2, label='LR_predict_value')
plt.yticks(range(0,6,2))
ax.set_ylabel('LR', fontsize=13)
ax.set_xlabel('Number_sample', fontsize=13)

# ax.plot(NL_input_data[:, 0],  'o-', color = 'blue', linewidth=2, label='DTS')
# ax.plot(NL_input_data[:, 1], 'o-',color = 'purple', linewidth=2, label='DT')
# plt.plot(Para_VVR[:, 2], a - 1, color = 'red', label = 'predict')
# plt.plot(Para_VVR[:, 2], a + 1, color = 'yellow', label = 'predict')
# plt.plot(Para_VVR[:, 2], a, label = 'predict')
# plt.xticks(np.arange(1, 38 , 1))
# plt.scatter(Onefil_Aim, a, s = 16)
# plt.plot(a, a)
plt.tick_params(labelsize=12)
# ax.set_xlabel(las_23.keys()[b])
#     # plt.savefig(r"D:\software\pycharm\PyCharm 1319.3.3\projects\predict_GASV\结果\onewey\图片\两口井3to5\onecon/" + name + ".png")
#     plt.savefig("lab_bp_loocv_carve_predict")

plt.show()

散点图

In [358]:
fig_2 = plt.figure(figsize=(20, 10))
fig_2.add_subplot(2, 3, 1)
plt.scatter(data[:,0], data[:,3], s = 16)
plt.xlabel("DTS/(us/m)",  fontsize=12)
plt.ylabel("GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
fig_2.add_subplot(2, 3, 2)
plt.scatter(data[:,1], data[:,3], s = 16)
plt.xlabel("DT/(us/m)", fontsize=12)
plt.ylabel("GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
fig_2.add_subplot(2, 3, 3)
plt.scatter(data[:,2], data[:,3], s = 16)
plt.plot(shuffle_RHOB, shuffle_LR_GAS + 1.6, color = 'blue')
plt.plot(shuffle_RHOB, shuffle_LR_GAS - 1.6, color = 'blue')
plt.xlabel("RHOB/(g/cm^3)", fontsize=12)
plt.ylabel("GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
fig_2.add_subplot(2, 3, 4)
plt.scatter(Onefil_data[:,0], Onefil_data[:,3], s = 16)
plt.xlabel("filtered_DTS/(us/m)", fontsize=12)
plt.ylabel("filtered_GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
fig_2.add_subplot(2, 3, 5)
plt.scatter(Onefil_data[:,1], Onefil_data[:,3], s = 16)
plt.xlabel("filtered_DT/(us/m)", fontsize=12)
plt.ylabel("filtered_GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
fig_2.add_subplot(2, 3, 6)
plt.scatter(Onefil_data[:,2], Onefil_data[:,3], s = 16)
plt.xlabel("filtered_RHOB/(g/cm^3)", fontsize=12)
plt.ylabel("filtered_GASmea/(cm^3/g)", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()